In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from IPython.display import Markdown
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [6]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
)

retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# we can set a similarity score threshold and only return documents with a score above that threshold.
# search_kwargs={"score_threshold": 0.5}
# We can also limit the number of documents k returned by the retriever.
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [7]:
def get_recipe(query, history=None):# completion llm  


    # 2. Incorporate the retriever into a question-answering chain.
    system_prompt = (
        """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Important instruction:
        When the user says phrases like "give me a recipe", "give me something to cook", or any variation of "give me" followed by a recipe request, interpret this as a request to *recommend* a recipe.
        Take the information from context[0]["metadata"]
        Then follow this format:
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    print(response)
    return response["answer"]

## Gradio App

In [8]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


## Test History

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


# Prompt para contextualizar la pregunta
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [10]:

system_prompt = (
      """
      Always add a kind and friendly message according to the context at the beginning of the response.
      Take the information from context[0]["metadata"]
      Then follow this format
      Title: [Title of the Recipe]
      Description: [Short Description of the Recipe]
      Preparation Time: [Preparation Time]
      Cooking Time: [Cooking Time]
      Difficulty: [Difficulty Level]
      Serves: [Number of Servings]
      Diet Type: [Diet Type]
      Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
      Ingredients: [List of Ingredients]
      Instructions: [Step-by-step Cooking Instructions]

      If you don't have any general information, just respond with "I don't know!"

      {context}
      {{metadata}}
      """
  )

qa_prompt = ChatPromptTemplate.from_messages(
  [
      ("system", system_prompt),
      MessagesPlaceholder("chat_history"),
      ("human", "{input}"),
  ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



### Stateful Management of chat history

In [11]:
from typing import Sequence

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "abc123"}}
def get_recipe(query, history=None):

  result = app.invoke(
    {"input": query},
    config=config,
)

  print("#### result", result["answer"])
  return result["answer"]

In [13]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli", "please recommend 3 recipes with rice"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [17]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

What can I do with brocolli
================================== Ai Message ==================================

Hello! Broccoli is such a versatile vegetable, and there are many delicious ways to use it. Here are a few ideas:

1. **Broccoli Soup**: Blend cooked broccoli with vegetable broth, cream, and seasonings for a creamy soup. Top with a cheesy crumble for added flavor.

2. **Roasted Broccoli**: Toss broccoli florets with olive oil, salt, and pepper, then roast in the oven until crispy. This makes a great side dish!

3. **Stir-Fry**: Add broccoli to your favorite stir-fry with other vegetables and protein for a quick and healthy meal.

4. **Salads**: Use raw or blanched broccoli in salads. Pair it with a vinaigrette made from capers, mustard, and honey for a tasty side.

5. **Falafel**: Use broccoli stalks to make falafel by blending them with chickpeas and spices, reducing food waste while creating a 

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage


def get_recipe(query, history=None):
    # Inicializa el historial de chat si es la primera vez
    history = []    # Prompt del sistema que guía cómo generar la respuesta
    
    system_prompt = (
        """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    # Configura el prompt del QA (sistema y humano)
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # Crear las cadenas para el QA y el RAG (retrieval-augmented generation)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # Invoca la cadena con la pregunta y el historial de chat
    response = rag_chain.invoke({"input": query, "chat_history": history})

    # Actualiza el historial de chat con los nuevos mensajes
    history.extend(
        [
            HumanMessage(content=query),
            AIMessage(content=response["answer"]),
        ]
    )

    # Imprimir el mensaje generado por la IA
    print("####### ai_msg_1:", response)

    # Devolver el contenido del mensaje de la IA
    return response["answer"]

In [ ]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli", "please recommend 3 recipes with rice"],
)

demo.launch()